In [2]:
from FuncionAzul import *

In [2]:
FuncionAzul(Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP_short1.csv", 
            Ruta_Nombre_Input_investigadores = "DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO_short1.csv",     
            Ruta_Nombre_Input_relaciones = "DATA_RELACION_ENTIDADES_MAESTRO_short1.csv",
            Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx', 
            Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", 
            Ruta_Nombre_Intermedio_cruce1_relaciones = "Test1/Nombre_Intermedio_cruce1_relaciones",
            Ruta_Nombre_Intermedio_cruce1_entidades = "Test1/Nombre_Intermedio_cruce1_entidades", 
            Ruta_Nombre_Intermedio_cruce2_invente = "Test1/Nombre_Intermedio_cruce2_invente", 
            Ruta_Nombre_Intermedio_cruce3_dir3 = "Test1/Nombre_Intermedio_cruce3_dir3",
            Ruta_Nombre_Output_investigadores = "Test1/Nombre_Output_investigadores", 
            Ruta_Nombre_Output_relaciones = "Test1/Ruta_Nombre_Output_relaciones",
            Ruta_Nombre_Output_entidades = "Test1/Ruta_Nombre_Output_entidades", 
            flag_invente_dir3 = False, 
            Flag_csv = False)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/02 12:28:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


22/08/02 12:28:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:35 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/08/02 12:28:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradat

22/08/02 12:28:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/02 12:28:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 1

22/08/02 12:28:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 12:28:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/02 1

22/08/02 12:29:13 ERROR Executor: Exception in task 1.0 in stage 28.0 (TID 92)
java.lang.OutOfMemoryError: GC overhead limit exceeded
	at net.jpountz.lz4.LZ4BlockOutputStream.<init>(LZ4BlockOutputStream.java:104)
	at org.apache.spark.io.LZ4CompressionCodec.compressedOutputStream(CompressionCodec.scala:144)
	at org.apache.spark.serializer.SerializerManager.wrapForCompression(SerializerManager.scala:159)
	at org.apache.spark.serializer.SerializerManager.wrapStream(SerializerManager.scala:134)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:163)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:306)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:171)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:99)
	at org.apache.spark.scheduler.ShuffleMapTask.run

Py4JJavaError: An error occurred while calling o685.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 28.0 failed 1 times, most recent failure: Lost task 1.0 in stage 28.0 (TID 92) (10.0.0.35 executor driver): java.lang.OutOfMemoryError: GC overhead limit exceeded
	at net.jpountz.lz4.LZ4BlockOutputStream.<init>(LZ4BlockOutputStream.java:104)
	at org.apache.spark.io.LZ4CompressionCodec.compressedOutputStream(CompressionCodec.scala:144)
	at org.apache.spark.serializer.SerializerManager.wrapForCompression(SerializerManager.scala:159)
	at org.apache.spark.serializer.SerializerManager.wrapStream(SerializerManager.scala:134)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:163)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:306)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:171)
	at org.apache.spark.shuffle.Shuff

22/08/02 12:29:16 WARN DiskBlockObjectWriter: Error deleting /private/var/folders/fy/cjds5kvd3692g8__1nlvytj00000gn/T/blockmgr-f4343148-b9de-49d1-befe-2e23650777ba/3e/temp_shuffle_d1c3f5ff-8cf5-4aa1-be09-1907e4ddd0ee
22/08/02 12:29:16 WARN DiskBlockObjectWriter: Error deleting /private/var/folders/fy/cjds5kvd3692g8__1nlvytj00000gn/T/blockmgr-f4343148-b9de-49d1-befe-2e23650777ba/3e/temp_shuffle_d0058680-7e06-4d50-aedb-ce109bf1a0b3
22/08/02 12:29:16 WARN DiskBlockObjectWriter: Error deleting /private/var/folders/fy/cjds5kvd3692g8__1nlvytj00000gn/T/blockmgr-f4343148-b9de-49d1-befe-2e23650777ba/0d/temp_shuffle_5f0df4c0-eb32-4c11-aeb3-dc9bfcd76ea8
22/08/02 12:29:16 WARN DiskBlockObjectWriter: Error deleting /private/var/folders/fy/cjds5kvd3692g8__1nlvytj00000gn/T/blockmgr-f4343148-b9de-49d1-befe-2e23650777ba/03/temp_shuffle_ba98ab0d-f9a8-439d-b010-8d2257a83994
22/08/02 12:29:16 WARN DiskBlockObjectWriter: Error deleting /private/var/folders/fy/cjds5kvd3692g8__1nlvytj00000gn/T/blockmgr-f4343

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 50128)
Traceback (most recent call last):
  File "/Users/arturo/opt/anaconda3/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/arturo/opt/anaconda3/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/arturo/opt/anaconda3/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/arturo/opt/anaconda3/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/arturo/opt/anaconda3/lib/python3.9/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/Users/arturo/opt/anaconda3/lib/python3.9/site-packages/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/Users/arturo/opt/an

In [3]:
FuncionAzul(Ruta_Nombre_Input_entidades = "DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP_short1.csv", 
            Ruta_Nombre_Input_investigadores = "DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO_short1.csv",     
            Ruta_Nombre_Input_relaciones = "DATA_RELACION_ENTIDADES_MAESTRO_short1.csv",
            Ruta_Nombre_Input_invente = 'Analisis_entidades_invente_15-02-2022.xlsx', 
            Ruta_Nombre_Input_dir3 = "DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", 
            Ruta_Nombre_Intermedio_cruce1_relaciones = "Test1/Nombre_Intermedio_cruce1_relaciones",
            Ruta_Nombre_Intermedio_cruce1_entidades = "Test1/Nombre_Intermedio_cruce1_entidades", 
            Ruta_Nombre_Intermedio_cruce2_invente = "Test1/Nombre_Intermedio_cruce2_invente", 
            Ruta_Nombre_Intermedio_cruce3_dir3 = "Test1/Nombre_Intermedio_cruce3_dir3",
            Ruta_Nombre_Output_investigadores = "Test1/Nombre_Output_investigadores", 
            Ruta_Nombre_Output_relaciones = "Test1/Ruta_Nombre_Output_relaciones",
            Ruta_Nombre_Output_entidades = "Test1/Ruta_Nombre_Output_entidades", 
            flag_invente_dir3 = True, 
            Flag_csv = True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/03 06:34:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


22/08/03 06:34:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:34:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:34:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:34:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:34:52 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/08/03 06:34:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradat

22/08/03 06:34:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:34:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:34:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:34:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:34:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:34:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 0

/Users/arturo/opt/anaconda3/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
100%|████████████████████████████████████████| 970/970 [00:06<00:00, 155.55it/s]


...

22/08/03 06:37:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


22/08/03 06:37:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 0

22/08/03 06:37:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 0

22/08/03 06:37:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 0

22/08/03 06:37:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 06:37:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/03 0